# Webscrapping

In [8]:
import pandas as pd

In [9]:
# Packages
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from urllib.error import HTTPError


#Pages Changing Url_Values 
values = [1, 51, 101, 151, 201]

#fileName CSV File
filename = "IMDB.csv"

#Opening 
f = open(filename, "w")


#Headers Used as Columns
headers = "ID, name, year, certificate, Genre, rating, metascore, Discription, Director, stars, Gross\n"
f.write(headers)

#Looping Through All Pages
for i in values:
    if i == 1:
        url_In = 'https://www.imdb.com/search/title/?groups=top_250&sort=user_rating'
    else:
        url_In = 'https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start={}&ref_=adv_nxt'.format(
            i)
    #List Of All Urls
    list_url = [url_In.format(i) for i in values]


    
    # OPENING UP CONNECTION ,GRABBING DATA
    try:
        uClient = uReq(url_In)
    except HTTPError as e:
        print(e)
    except URLError as e:
        print('The Server Not Found')
    else:
        page_html = uClient.read()
        uClient.close()

    # HTML PARSING
    page_soup = soup(page_html.decode('utf-8','ignore'), "html.parser")

    # containers
    containers = page_soup.findAll("div", {"class": "lister-item mode-advanced"})


    # LOPPING in Each containers of Cards

    for item in containers:
        item_content = item.find('div', {'class': "lister-item-content"})
        # index
        ID = int(item_content.h3.span.text.split('.')[0])

        # Movie
        #name = item_content.h3.a.text
        name = item_content.h3.a.string
        print(name)


        # year
        try:
            year = int(item_content.h3.find('span', {'class': "lister-item-year text-muted unbold"}).text[1:5])
        except ValueError:
            year = int(item_content.h3.find('span', {'class': "lister-item-year text-muted unbold"}).text[5:9])

   # Other colmns Data for gener , certificate , Time
        block1 = item_content.find('p', {'class': 'text-muted'})

        # certificate
        if block1.find('span', {'class': "certificate"}):
            certificate = block1.find('span', {'class': "certificate"}).text
        else:
            certificate = None

        # Genre
        Genre = block1.find('span', {'class': "genre"}).text.replace('\n', '')

        # BLOCK FOR RATING METASCORE
        block2 = item_content.find('div', {'class': 'ratings-bar'})
        
        # Rating
        rating = float(block2.div.strong.text)
        
        # Metascore
        if block2.find(class_='inline-block ratings-metascore'):
            metascore = int(block2.find('div', {'class': 'inline-block ratings-metascore'}).span.text)
        else:
            metascore = None
        
        # Description
        Discription = (item_content.findAll('p', {'class': 'text-muted'})[1]).text.replace('\n', '')

        # Director
        block3 = item_content.find('p', {'class': ""}).find_all('a')
        Director = block3[0].text.replace(',','|')

        # stars
        stars = ",".join(block3[i].text for i in range(1, len(block3)))
        
        # Gross
        try:
            Gross = (item_content.find('p', {'class': "sort-num_votes-visible"}).findAll('span')[4]).text
        except IndexError:
            Gross =None
        

        f.write(
            '{},{},{},{},{},{},{},{},{},{},{}\n'.format(ID, name.replace(',', '.'), year, certificate,
                                                     Genre.replace(',', '|'),
                                                     rating, metascore, Discription.replace(',', '.'), Director,
                                                     stars.replace(',', ' |'),Gross))
                                                     
f.close()



The Shawshank Redemption
The Godfather
The Dark Knight
The Godfather: Part II
The Lord of the Rings: The Return of the King
Pulp Fiction
Schindler's List
12 Angry Men
Inception
Fight Club
The Lord of the Rings: The Fellowship of the Ring
Forrest Gump
Il buono, il brutto, il cattivo
1917
The Lord of the Rings: The Two Towers
The Matrix
Goodfellas
Star Wars: Episode V - The Empire Strikes Back
One Flew Over the Cuckoo's Nest
Seppuku
Joker
Gisaengchung
Interstellar
Cidade de Deus
Sen to Chihiro no kamikakushi
Saving Private Ryan
The Green Mile
La vita è bella
Se7en
The Silence of the Lambs
Star Wars
Shichinin no samurai
It's a Wonderful Life
Avengers: Endgame
Avengers: Infinity War
Whiplash
The Intouchables
The Prestige
The Departed
The Pianist
Gladiator
American History X
The Usual Suspects
Léon
The Lion King
Terminator 2: Judgment Day
Nuovo Cinema Paradiso
Hotaru no haka
Back to the Future
Once Upon a Time in the West
Tengoku to jigoku
Psycho
Casablanca
The Great Dictator
Modern Times
C

In [10]:
data = pd.read_csv('IMDB.csv')

In [11]:
data.head()

,ID,name,year,certificate,Genre,rating,metascore,Discription,Director,stars,Gross
0,1,The Shawshank Redemption,1994,A,Drama,9.3,80,Two imprisoned men bond over a number of y...,Frank Darabont,Tim Robbins |Morgan Freeman |Bob Gunton |Willi...,$28.34M
1,2,The Godfather,1972,A,Crime| Drama,9.2,100,The aging patriarch of an organized crime ...,Francis Ford Coppola,Marlon Brando |Al Pacino |James Caan |Diane Ke...,$134.97M
2,3,The Dark Knight,2008,UA,Action| Crime| Drama,9.0,84,When the menace known as the Joker wreaks ...,Christopher Nolan,Christian Bale |Heath Ledger |Aaron Eckhart |M...,$534.86M
3,4,The Godfather: Part II,1974,None,Crime| Drama,9.0,90,The early life and career of Vito Corleone...,Francis Ford Coppola,Al Pacino |Robert De Niro |Robert Duvall |Dian...,$57.30M
4,5,The Lord of the Rings: The Return of the King,2003,16,Adventure| Drama| Fantasy,8.9,94,Gandalf and Aragorn lead the World of Men ...,Peter Jackson,Elijah Wood |Viggo Mortensen |Ian McKellen |Or...,$377.85M
